In [60]:
# all the imports
from pathlib import Path
import datetime
import pandas as pd
from report import *
import datetime

In [61]:
curr_dir = Path.cwd()
data_dir = curr_dir / 'User_data'
log_file_path = []

# get the latest log file
for user in data_dir.iterdir():
    for session in user.iterdir():
        for log in session.iterdir():
            if log.suffix == '.log':
                log_file_path.append(log)


In [62]:
file_path = log_file_path[0]
print(file_path)

/home/niskal/Abhishek/OpenBCI_DataAnlaysis/User_data/12345/session_2/12345_session_2_22-03-2023_21-59.log


In [63]:
comm_list = []
sysmon_list = []
start_time = 0
end_time = 0

In [64]:
with open(file_path, 'r') as f:
    lines  = f.readlines()[2:]

start_time = lines[0].split()[0]
end_time = lines[-1].split()[0]
f.close()

In [65]:
# traverse thorugh the log file for sysmon and communication events
with open(file_path ,'r') as f:
    for line in f:
        line = line.split('\t')

        if(len(line)<=1):
            continue

        if line[2] == 'SYSMON':
            sysmon_list.append(line)

        if line[2] == 'COMMUN' and line[6]!='SELECTED\n':
            comm_list.append(line)

In [66]:
def create_dict(start_time, end_time):
    #make it in HH:MM:SS format
    start_time = start_time.split('.')[0]
    start = datetime.datetime.strptime(start_time, '%H:%M:%S')
    end_time = end_time.split('.')[0]
    end = datetime.datetime.strptime(end_time, '%H:%M:%S')
    time_diff = end - start
    time_list = [start + datetime.timedelta(seconds=x) for x in range(time_diff.seconds + 1)]
    time_dict = {time.strftime('%H:%M:%S'): 0 for time in time_list}
    return time_dict

In [67]:
time_dict  = create_dict(start_time, end_time)
getSysmonScore(sysmon_list, time_dict)

#print where time dict has value 1
for key, value in time_dict.items():
    if value == 1:
        print(key)

1.038313
0.835807
0.895221
0.694475
0.762519
['21:56:36.572304', '', 'SYSMON', 'ACTION', 'D', 'MISS\n']
0.913812
1.28743
0.872888
0.832487
['21:57:14.147796', '', 'SYSMON', 'ACTION', 'D', 'MISS\n']
1.360443
0.984618
1.382973
['21:57:44.723766', '', 'SYSMON', 'ACTION', 'A', 'MISS\n']
2.027664
['21:58:07.778038', '', 'SYSMON', 'ACTION', 'E', 'MISS\n']
2.001091
1.359702
0.990621
1.562321
Total Failures:  17
Total Hits:  13
False Alarms:  4
Average Response Time:  1.1648461764705882
